In [1]:
from langchain_groq import ChatGroq
from config import groq_api_key
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = groq_api_key,
    model_name = "llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
loader = WebBaseLoader('https://jobs.citi.com/job/chennai/officer-business-data-analyst-c11-hybrid-chennai/287/71351588688?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic')
page_data = loader.load().pop().page_content
print(page_data)










Officer-Business Data Analyst-C11-Hybrid-Chennai at Citi

















































































































Skip to main content







Home



Home
Working at Citi is far more than just a job. At Citi, you’ll have the opportunity to grow your career, give back to your community and make a real impact.





Search & Apply
Create/Manage Profile
Upload Resume & Match Me
Internal Career Mobility
Alumni





Locations



Locations
Around the world, Citi has strategic locations where our employees go to work each day to support our core businesses and drive solutions for our clients.




Asia Pacific
Europe, Middle East, and Africa
Latin America
North America






Why Citi



Why Citi
Working at Citi is far more than just a job. A career with us means joining a team of more than 200,000 dedicated people from around the globe.




Our Values
Global Benefits


Global Diversity and Inclusion
Citi Salutes


About Us


Teams


In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
  {
    "role": "Officer-Business Data Analyst-C11-Hybrid-Chennai",
    "experience": "4-8 Years",
    "skills": [
      "Excellent analytical & reasoning skills",
      "Working experience in BFSI is required – corporate preferred",
      "Sound knowledge of query languages SQL and / or others",
      "Working experience in JSON / other messages",
      "Good understanding of different data modelling approaches",
      "Sound knowledge of big data & cloud ecosystem"
    ],
    "description": "Formulate and define systems scope and project objectives through research activities. Analyze business client needs and document requirements by utilizing business analysis procedures and concepts. Assess applicability of previous or similar experiences and evaluate options under circumstances not covered by procedures or precedents. Identify risk and consider business implications of the application of technology to the current business environment. Create and prepare reports, metrics and pre

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


[{'role': 'Officer-Business Data Analyst-C11-Hybrid-Chennai',
  'experience': '4-8 Years',
  'skills': ['Excellent analytical & reasoning skills',
   'Working experience in BFSI is required – corporate preferred',
   'Sound knowledge of query languages SQL and / or others',
   'Working experience in JSON / other messages',
   'Good understanding of different data modelling approaches',
   'Sound knowledge of big data & cloud ecosystem'],
  'description': "Formulate and define systems scope and project objectives through research activities. Analyze business client needs and document requirements by utilizing business analysis procedures and concepts. Assess applicability of previous or similar experiences and evaluate options under circumstances not covered by procedures or precedents. Identify risk and consider business implications of the application of technology to the current business environment. Create and prepare reports, metrics and presentations. Appropriately assess risk whe

In [6]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:
import uuid
import chromadb
from chromadb.utils import embedding_functions

# Example embedding function
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-MiniLM-L6-v2")

client = chromadb.PersistentClient(path='vectorstore')
collection = client.get_or_create_collection(name="portfolio", embedding_function=embedding_function)

# Add documents to the collection
if collection.count() == 0:
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas=[{"links": row["Links"]}],
            ids=[str(uuid.uuid4())]
        )


/home/ntejha/anaconda3/envs/yoda/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-17 13:59:26.152869: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 13:59:26.162025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 13:59:26.173155: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has 

In [8]:
job = json_res[0]
job

{'role': 'Officer-Business Data Analyst-C11-Hybrid-Chennai',
 'experience': '4-8 Years',
 'skills': ['Excellent analytical & reasoning skills',
  'Working experience in BFSI is required – corporate preferred',
  'Sound knowledge of query languages SQL and / or others',
  'Working experience in JSON / other messages',
  'Good understanding of different data modelling approaches',
  'Sound knowledge of big data & cloud ecosystem'],
 'description': "Formulate and define systems scope and project objectives through research activities. Analyze business client needs and document requirements by utilizing business analysis procedures and concepts. Assess applicability of previous or similar experiences and evaluate options under circumstances not covered by procedures or precedents. Identify risk and consider business implications of the application of technology to the current business environment. Create and prepare reports, metrics and presentations. Appropriately assess risk when busines

In [9]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [11]:


prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Tejha, a student at SRM. Tejha is an AI & Software Expert dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over his experience, he have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Tejha 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Tejha portfolio: {link_list}
        Remember you are Tejha, Student at SRMIST. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)



Subject: Application for Officer-Business Data Analyst Position at Citigroup

Dear Hiring Manager,

I am excited to apply for the Officer-Business Data Analyst position at Citigroup, as advertised. With my strong analytical and technical skills, I am confident that I can make a valuable contribution to your team.

As a student at SRMIST with a strong background in AI and software development, I possess excellent analytical and reasoning skills, which I believe are essential for this role. My experience in working with various technologies, including SQL, JSON, and big data ecosystems, aligns well with the job requirements. Additionally, my understanding of different data modeling approaches and experience in creating reports, metrics, and presentations will enable me to effectively communicate insights and recommendations to stakeholders.

I am particularly drawn to this role because of the opportunity to work in the BFSI sector, where I can apply my skills and knowledge to drive busin